IMAGE


In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import f1_score,classification_report
from sklearn.ensemble import ExtraTreesClassifier
import os
from IPython.core.display import HTML, display
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
%matplotlib inline
from skimage import color
from skimage.io import imread, imshow
from skimage.transform import resize
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from nltk.corpus import stopwords

C:\Users\NAWAB-PC\AppData\Local\Temp\ipykernel_12112\3773849246.py:18: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import HTML, display


In [2]:
from os import listdir

images = listdir(r"C:\Users\NAWAB-PC\Downloads\final project\images")

len(images)

6992

In [ ]:
#later
# number=1
#     path=f"image_{number}"
#     number=+1
#     print(number)
#     try:
#         image1 = imread(r"C:\Users\NAWAB-PC\Downloads\final project\images\\"+ path + r".jpg",as_gray=True)
#         resized_img = resize(image1, (10,10))
#         pic_dict[path]=resized_img
#     except FileNotFoundError:
#         image1 = imread(r"C:\Users\NAWAB-PC\Downloads\final project\images\\"+ path + r".jpeg",as_gray=True)
#         resized_img = resize(image1, (10,10))
#         pic_dict[path]=resized_img
#     if number==5:
#         break
# imshow(image1);
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract"
try:
    text=pytesseract.image_to_string(r"C:\Users\NAWAB-PC\Downloads\final project\images\\"+ path + r".jpg")
    
except :
     text=pytesseract.image_to_string(r"C:\Users\NAWAB-PC\Downloads\final project\images\\"+ path + r".jpeg")

In [3]:
import re
def sorting(string):
    return list(map(int, re.findall(r'\d+', string)))[0]
images.sort(key = sorting)

In [ ]:
from tqdm.notebook import tqdm #progress bar
pic_dict={}
for i in tqdm(images):
    img=imread(r"C:\Users\NAWAB-PC\Downloads\final project\images\\" + i, as_gray=True)
    resized_img = resize(img, (256,128))
    pic_dict[i]=resized_img

In [6]:
import pickle
o=open("project.pkl","wb")
pickle.dump(pic_dict,o)
o.close()

In [2]:
import pickle
o=open("project.pkl","rb")
data=pickle.load(o)
o.close()

In [298]:
len(data)

6992

In [265]:
from skimage.feature import hog
from sklearn.feature_extraction import image
patch_f={}
for i in data.keys():
#     fd, hog_image = hog(data[i], orientations=9, pixels_per_cell=(8, 8),
#                 cells_per_block=(2, 2), visualize=True, multichannel=True)
    patches = image.extract_patches_2d(data[i], (2, 2))
    patch_f[i]=patches

In [3]:
from skimage import filters
from skimage import feature
from sklearn.preprocessing import MinMaxScaler
scaling=MinMaxScaler()
canny_dict={}
for i in data.keys():
    scale=scaling.fit_transform(data[i])
    can = feature.canny(scale)
    canny_dict[i]=can.ravel()

In [4]:
import imageio
import imgaug as aug
import imgaug.augmenters as augmenters
canny_features={}
canny_features1={}
scaling=MinMaxScaler()
for i in data.keys():
    
    rotate_1=augmenters.Affine(rotate=(-40,25))
    scale=scaling.fit_transform(data[i])
    input_rotate_1 = rotate_1.augment_image(scale)
    
    canny_f=feature.canny(input_rotate_1)
    canny_features[i]=canny_f.ravel()
    #next
    rotate_2=augmenters.Affine(rotate=(55,25))
    scale=scaling.fit_transform(data[i])
    input_rotate_2 = rotate_2.augment_image(scale)
    
    canny_f=feature.canny(input_rotate_2)
    canny_features1[i]=canny_f.ravel()
    

In [5]:
len(canny_dict.keys())

6992

In [69]:
filename='minmaxscaler.pkl'
pickle.dump(scaling, open(filename, 'wb'))

In [6]:
original_features=pd.DataFrame(canny_dict)
df1=pd.DataFrame(canny_features)


In [7]:
df1=df1.T

original_features=original_features.T

In [8]:
df3=pd.DataFrame(canny_features1)

In [9]:
df3=df3.T

In [10]:
original_features=original_features.drop(['image_120.jpg', 'image_4800.jpg','image_6782.jpg','image_6785.jpg','image_6787.jpg'])

In [11]:
df1=df1.drop(['image_120.jpg', 'image_4800.jpg','image_6782.jpg','image_6785.jpg','image_6787.jpg'])

In [12]:
df3=df3.drop(['image_120.jpg', 'image_4800.jpg','image_6782.jpg','image_6785.jpg','image_6787.jpg'])

In [13]:
main_data=pd.read_csv(r"C:\Users\NAWAB-PC\Downloads\final project\labels.csv")
del main_data['Unnamed: 0']
main_data=main_data[main_data['text_corrected'].notnull()]
sentiments=main_data['overall_sentiment']
le = LabelEncoder()
y = le.fit_transform(sentiments)

In [14]:
#verypositive=4
#positive=2
#neutral=1
#negative=0
#verynegative=3
df1['sentiment']=y
df3['sentiment']=y
original_features['sentiment']=y


In [15]:
lst=[]
for i in df1['sentiment']:
    
    if i==3:
        i=0
        lst.append(i)
    elif i==4:
        i=2
        lst.append(i)
    else:
        lst.append(i)

In [16]:
lst=[]
for i in df3['sentiment']:
    
    if i==3:
        i=0
        lst.append(i)
    elif i==4:
        i=2
        lst.append(i)
    else:
        lst.append(i)

In [17]:
lst=[]
for i in original_features['sentiment']:
    
    if i==3:
        i=0
        lst.append(i)
    elif i==4:
        i=2
        lst.append(i)
    else:
        lst.append(i)

In [18]:
df1['sentiment']=lst
df3['sentiment']=lst
original_features['sentiment']=lst

In [19]:
negative_=df1.groupby('sentiment').get_group(0)


In [20]:
df1.groupby('sentiment').get_group(0)

,0,1,2,3,4,5,6,7,8,9,...,32759,32760,32761,32762,32763,32764,32765,32766,32767,sentiment
image_6.jpg,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0
image_7.png,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0
image_11.jpg,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0
image_15.jpg,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0
image_50.jpg,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
image_6961.jpg,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0
image_6962.jpg,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0
image_6966.png,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0
image_6972.jpg,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0


In [21]:
len(negative_)

631

In [22]:
negative_rot2=df1[(df3['sentiment']==0)]


In [23]:
concatlist=[negative_,negative_rot2,original_features]
finalconcat=pd.concat(concatlist)

In [24]:
from sklearn.utils import shuffle
finalconcat = shuffle(finalconcat,random_state=100)

In [25]:
X=finalconcat.drop(['sentiment'],axis=1)

y=(finalconcat['sentiment'])

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=44)

MODELS

In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
exp = LogisticRegression(solver='saga',C=33.0,fit_intercept=True,intercept_scaling=1,tol=0.00001,random_state=42,max_iter=34).fit(X_train,y_train)
y_pred = exp.predict(X_test)

print('Accuracy Score : ' + str(accuracy_score(y_test,y_pred)))
print("F1 Score: ",f1_score(y_test, y_pred,average='macro'))
print('Confusion Matrix : \n' + str(confusion_matrix(y_test,y_pred)))
print(classification_report(y_pred,y_test))

C:\Users\NAWAB-PC\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Accuracy Score : 0.5761616161616162
F1 Score:  0.5499792765310533
Confusion Matrix : 
[[374  61 130]
 [ 33 160 456]
 [ 55 314 892]]
              precision    recall  f1-score   support

           0       0.66      0.81      0.73       462
           1       0.25      0.30      0.27       535
           2       0.71      0.60      0.65      1478

    accuracy                           0.58      2475
   macro avg       0.54      0.57      0.55      2475
weighted avg       0.60      0.58      0.58      2475



In [137]:
import pickle
modelpkl4 = "logistic.pkl"  
with open(modelpkl4, 'wb') as file:  
    pickle.dump(exp, file)

In [28]:
from sklearn.linear_model import SGDClassifier

sgd = SGDClassifier(random_state=42, max_iter=1000, tol=1e-3,loss='log')
sgd.fit(X_train, y_train)
pred8=sgd.predict(X_test)
print("F1 Score: ",f1_score(y_test, pred8,average='macro'))
print(classification_report(pred8,y_test))

F1 Score:  0.5320374156534541
              precision    recall  f1-score   support

           0       0.68      0.71      0.69       541
           1       0.24      0.30      0.27       512
           2       0.68      0.60      0.64      1422

    accuracy                           0.56      2475
   macro avg       0.53      0.54      0.53      2475
weighted avg       0.59      0.56      0.57      2475



In [138]:
import pickle
modelpkl5 = "sgdc.pkl"  
with open(modelpkl5, 'wb') as file:  
    pickle.dump(sgd, file)

In [29]:
print('Confusion Matrix : \n' + str(confusion_matrix(y_test,pred8)))

Confusion Matrix : 
[[382  53 130]
 [ 55 156 438]
 [104 303 854]]


In [39]:
knn2=KNeighborsClassifier(n_neighbors=9,weights='distance',leaf_size=9,algorithm='brute')
knn2.fit(X_train,y_train)
y_pred_knn2=knn2.predict(X_test)
accuracy_score(y_pred_knn2,y_test)
print(classification_report(y_pred_knn2,y_test))
print("F1 Score: ",f1_score(y_test, y_pred_knn2,average='macro'))
print('Confusion Matrix : \n' + str(confusion_matrix(y_test,y_pred_knn2)))

              precision    recall  f1-score   support

           0       0.65      0.69      0.67       537
           1       0.23      0.31      0.27       480
           2       0.70      0.60      0.65      1458

    accuracy                           0.57      2475
   macro avg       0.53      0.54      0.53      2475
weighted avg       0.60      0.57      0.58      2475

F1 Score:  0.5281750100497588
Confusion Matrix : 
[[370  57 138]
 [ 59 150 440]
 [108 273 880]]


In [142]:
import pickle
modelpkl6 = "knn2.pkl"  
with open(modelpkl6, 'wb') as file:  
    pickle.dump(knn2, file)

In [41]:
from sklearn.ensemble import ExtraTreesClassifier
clf5 = ExtraTreesClassifier(n_estimators=10, max_depth=None,min_samples_split=2, random_state=0)
clf5.fit(X_train, y_train)

clf_p=clf5.predict(X_test)
accuracy_score(clf_p,y_test)
print(classification_report(clf_p,y_test))
print("F1 Score: ",f1_score(y_test, clf_p,average='macro'))
print('Confusion Matrix : \n' + str(confusion_matrix(y_test,clf_p)))

              precision    recall  f1-score   support

           0       0.68      0.80      0.73       479
           1       0.24      0.29      0.27       534
           2       0.69      0.60      0.64      1462

    accuracy                           0.57      2475
   macro avg       0.54      0.56      0.55      2475
weighted avg       0.59      0.57      0.58      2475

F1 Score:  0.5468785509902978
Confusion Matrix : 
[[382  52 131]
 [ 37 157 455]
 [ 60 325 876]]


In [46]:
import pickle
modelpkl7 = "ex.pkl"  
with open(modelpkl7, 'wb') as file:  
    pickle.dump(clf5, file)

In [ ]:
from sklearn.ensemble import VotingClassifier
voting = VotingClassifier(
estimators=[('ex',clf5),("sgd",sgd),('lr',exp)],
voting='hard')
voting.fit(X_train,y_train)
pred_final=voting.predict(X_test)
print('Accuracy:', accuracy_score(y_test, pred_final))
print("F1 Score: ",f1_score(y_test, pred_final,average='macro'))

In [ ]:
from skimage.feature import hog
# form skimage import feature
fd_dict={}
hog_dict={}
hog_features={}
for i in data.keys():
    if(data[i].shape==(256, 128, 3) or data[i].shape==(256,128,4)):
        fd, hog_image = hog(data[i], orientations=9, pixels_per_cell=(8, 8),
                cells_per_block=(2, 2), visualize=True, multichannel=True)
        
        fd_dict[i]=fd
        hog_dict[i]=hog_image
        rotate_1=augmenters.Affine(rotate=(-50,20))
        input_rotate_1 = rotate_1.augment_image(data[i])
        images_list=[data[i], input_rotate_1]
        hog_i=hog(input_rotate_1)
        hog_features[i]=hog_i.flatten()
    else:
        fd, hog_image = hog(data[i], orientations=9, pixels_per_cell=(8, 8),
                cells_per_block=(2, 2), visualize=True, multichannel=False)
        fd_dict[i]=fd
        hog_dict[i]=hog_image
        rotate_1=augmenters.Affine(rotate=(-50,20))
        input_rotate_1 = rotate_1.augment_image(data[i])
        images_list=[data[i], input_rotate_1]
        hog_i=hog(input_rotate_1)
        hog_features[i]=hog_i.flatten()

In [23]:
main_data

,image_name,text_ocr,text_corrected,overall_sentiment
0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,very_positive
1,image_2.jpeg,The best of #10 YearChallenge! Completed in le...,The best of #10 YearChallenge! Completed in le...,very_positive
2,image_3.JPG,Sam Thorne @Strippin ( Follow Follow Saw every...,Sam Thorne @Strippin ( Follow Follow Saw every...,positive
3,image_4.png,10 Year Challenge - Sweet Dee Edition,10 Year Challenge - Sweet Dee Edition,positive
4,image_5.png,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,neutral
...,...,...,...,...
6987,image_6988.jpg,Tuesday is Mardi Gras Wednesday is Valentine's...,Tuesday is Mardi Gras Wednesday is Valentine's...,neutral
6988,image_6989.jpg,MUST WATCH MOVIES OF 2017 ITI Chennai memes MA...,MUST WATCH MOVIES OF 2017 ITI Chennai memes MA...,neutral
6989,image_6990.png,LESS MORE TALKING PLANNING SODA JUNK FOOD COMP...,LESS MORE TALKING PLANNING SODA JUNK FOOD COMP...,positive
6990,image_6991.jpg,When I VERY have time is a fantasy No one has ...,When I have time is a fantasy. no one has time...,very_positive


In [24]:
import string
import nltk
from gensim.parsing.preprocessing import remove_stopwords
import re
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
t2=[]
PS = PorterStemmer()
for i in main_data['text_corrected']:
    sen=(i.lower())
    sen=(re.sub("[^A-Za-z ]",' ',sen))
    sen=remove_stopwords(sen)
    t2.append(sen)

clean=[]
for i in t2:
    word_list=word_tokenize(i)
    updated=[]
    for j in word_list:
        updated.append(PS.stem(j))
    clean.append(" ".join(updated))

In [42]:
import texthero
main_data['cleaned']=texthero.clean(main_data['text_corrected'])

In [43]:
main_data['text_corrected'].isnull().sum()

0

In [79]:
nan_idx=main_data[main_data['text_corrected'].isnull()].index.tolist()
nan_idx


[119, 4799, 6781, 6784, 6786]

In [47]:
#3 labels
lst=[]
for i in main_data['overall_sentiment']:
    
    if i=='very_positive':
        i='positive'
        lst.append(i)
    elif i=='very_negative':
        i='negative'
        lst.append(i)
    else:
        lst.append(i)

In [45]:
main_data['overall_sentiment']=lst

In [48]:
main_data

,image_name,text_ocr,text_corrected,overall_sentiment,cleaned
0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,positive,look friend lightyear sohalikut trend play yea...
1,image_2.jpeg,The best of #10 YearChallenge! Completed in le...,The best of #10 YearChallenge! Completed in le...,positive,best yearchallenge completed less years kudus ...
2,image_3.JPG,Sam Thorne @Strippin ( Follow Follow Saw every...,Sam Thorne @Strippin ( Follow Follow Saw every...,positive,sam thorne strippin follow follow saw everyone...
3,image_4.png,10 Year Challenge - Sweet Dee Edition,10 Year Challenge - Sweet Dee Edition,positive,year challenge sweet dee edition
4,image_5.png,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,neutral,year challenge filter hilarious year challenge...
...,...,...,...,...,...
6987,image_6988.jpg,Tuesday is Mardi Gras Wednesday is Valentine's...,Tuesday is Mardi Gras Wednesday is Valentine's...,neutral,tuesday mardi gras wednesday valentine friday ...
6988,image_6989.jpg,MUST WATCH MOVIES OF 2017 ITI Chennai memes MA...,MUST WATCH MOVIES OF 2017 ITI Chennai memes MA...,neutral,must watch movies iti chennai memes maanagaram...
6989,image_6990.png,LESS MORE TALKING PLANNING SODA JUNK FOOD COMP...,LESS MORE TALKING PLANNING SODA JUNK FOOD COMP...,positive,less talking planning soda junk food complaini...
6990,image_6991.jpg,When I VERY have time is a fantasy No one has ...,When I have time is a fantasy. no one has time...,positive,time fantasy one time unless make time arhtist...


In [49]:
main_data['overall_sentiment'].unique()

array(['positive', 'neutral', 'negative'], dtype=object)

In [50]:
import nlpaug
import nlpaug.augmenter.word as nlp
aug = nlp.SynonymAug(aug_src='wordnet',aug_max=2)
aug_dict={}
idx=1
for i in main_data['cleaned']:
    sen=aug.augment(i,n=2)
    aug_dict[idx]=sen
    idx+=1

In [105]:
main_data.head(350)

,image_name,text_ocr,text_corrected,overall_sentiment,cleaned
0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,very_positive,look friend lightyear sohalikut trend play yea...
1,image_2.jpeg,The best of #10 YearChallenge! Completed in le...,The best of #10 YearChallenge! Completed in le...,very_positive,best yearchallenge completed less years kudus ...
2,image_3.JPG,Sam Thorne @Strippin ( Follow Follow Saw every...,Sam Thorne @Strippin ( Follow Follow Saw every...,positive,sam thorne strippin follow follow saw everyone...
3,image_4.png,10 Year Challenge - Sweet Dee Edition,10 Year Challenge - Sweet Dee Edition,positive,year challenge sweet dee edition
4,image_5.png,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,neutral,year challenge filter hilarious year challenge...
...,...,...,...,...,...
346,image_347.png,Marvel Memes TELLS EVERYONE HE CAN SEE EVERYTH...,Marvel Memes TELLS EVERYONE HE CAN SEE EVERYTH...,very_positive,marvel memes tells everyone see everything kno...
347,image_348.jpg,Doctor Strange: don't worry guys Doctor Strang...,Doctor Strange: don't worry guys Doctor Strang...,negative,doctor strange worry guys doctor strange plan ...
348,image_349.png,it's literally that meme,it's literally that meme,neutral,literally meme
349,image_350.jpg,I KNOW IT'S SOAP BUT IT LOOKS DELICIOUS,I KNOW IT'S SOAP BUT IT LOOKS DELICIOUS,neutral,know soap looks delicious


In [51]:
negative_text=main_data.groupby('overall_sentiment').get_group('negative')


In [52]:
negative_text


,image_name,text_ocr,text_corrected,overall_sentiment,cleaned
5,image_6.jpg,"1998: ""Don't get in car with strangers"" 2008: ...","1998: ""Don't get in car with strangers"" 2008: ...",negative,get car strangers meet people internet alone m...
6,image_7.png,10 years challenge is lit af Üs & B erg 1945 1...,10 years challenge is lit af Üs & B erg 1945 1...,negative,years challenge lit af us b erg wow transforma...
10,image_11.jpg,PROBABLY THE FIRST MAN TO DO 10 YEAR CHALLENGE...,PROBABLY THE FIRST MAN TO DO 10 YEAR CHALLENGE...,negative,probably first man year challenge chennai pasa...
14,image_15.jpg,Anti-vaxx kids when they see someone doing the...,Anti-vaxx kids when they see someone doing the...,negative,anti vaxx kids see someone year challenge
49,image_50.jpg,Hawkeye is important in Infinity War Is he tho...,Hawkeye is important in Infinity War Is he tho...,negative,hawkeye important infinity war though
...,...,...,...,...,...
6960,image_6961.jpg,THAT FACE YOUR FRIEND GIVES YOU WHEN YOUR CRUS...,THAT FACE YOUR FRIEND GIVES YOU WHEN YOUR CRUS...,negative,face friend gives crush walks room
6961,image_6962.jpg,NIGHTMARES Are made of this.,NIGHTMARES Are made of this.,negative,nightmares made
6965,image_6966.png,Baby Mr. Bean Funny Picture,Baby Mr. Bean Funny Picture,negative,baby mr bean funny picture
6971,image_6972.jpg,YOUR FACE MemeCenter.com WHEN THERES A SUBSTIT...,YOUR FACE MemeCenter.com WHEN THERES A SUBSTIT...,negative,face memecenter com theres substitute math class


In [53]:
aug = nlp.SynonymAug(aug_src='wordnet',aug_max=2)
neg_dict={}
idx=1
for i in negative_text['cleaned']:
    sen=aug.augment(i,n=1)
    neg_dict[idx]=sen
    idx+=1

In [54]:
concatlist_text=[negative_text,main_data]
finalconcat_text=pd.concat(concatlist_text)

In [55]:
finalconcat_text['overall_sentiment'].value_counts()

positive    4156
neutral     2200
negative    1262
Name: overall_sentiment, dtype: int64

In [56]:
from sklearn.utils import shuffle
finalconcat_text = shuffle(finalconcat_text,random_state=100)

In [57]:
#exper
X=finalconcat_text['cleaned']

y=finalconcat_text['overall_sentiment']
le = LabelEncoder()
y = le.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=44)


In [58]:
#exp
vectorizer = CountVectorizer()
v1 = vectorizer.fit_transform(X_train)
v2 = vectorizer.transform(X_test)
#direct model iske baad

In [68]:
filename='countvector.pkl'
pickle.dump(vectorizer, open(filename, 'wb'))

MODELS

In [59]:
clf = DecisionTreeClassifier(criterion='gini',splitter='best',random_state=46,max_features=6)
clf.fit(v1,y_train)
y_pred=clf.predict(v2)
print('Accuracy:', accuracy_score(y_test, y_pred))
print("F1 Score: ",f1_score(y_test, y_pred,average='macro')) 
print(classification_report(y_test, y_pred))

Accuracy: 0.5393700787401575
F1 Score:  0.5303582604945697
              precision    recall  f1-score   support

           0       0.57      0.73      0.64       373
           1       0.36      0.32      0.34       681
           2       0.62      0.60      0.61      1232

    accuracy                           0.54      2286
   macro avg       0.52      0.55      0.53      2286
weighted avg       0.53      0.54      0.53      2286



In [60]:
print('Confusion Matrix : \n' + str(confusion_matrix(y_test,y_pred)))

Confusion Matrix : 
[[272  40  61]
 [ 70 221 390]
 [135 357 740]]


In [61]:
knn=KNeighborsClassifier(n_neighbors=13,weights='distance',leaf_size=9,algorithm='auto')
knn.fit(v1,y_train)
y_pred_knn=knn.predict(v2)
accuracy_score(y_pred_knn,y_test)
print(classification_report(y_pred_knn,y_test))
print("F1 Score: ",f1_score(y_test, y_pred_knn,average='macro'))

              precision    recall  f1-score   support

           0       0.70      0.94      0.80       277
           1       0.05      0.34      0.08        91
           2       0.95      0.61      0.74      1918

    accuracy                           0.64      2286
   macro avg       0.56      0.63      0.54      2286
weighted avg       0.88      0.64      0.72      2286

F1 Score:  0.5414792883186667


In [62]:
print('Confusion Matrix : \n' + str(confusion_matrix(y_test,y_pred_knn)))

Confusion Matrix : 
[[ 260   12  101]
 [   5   31  645]
 [  12   48 1172]]


In [63]:
from sklearn.linear_model import RidgeClassifier
rc = RidgeClassifier(solver='auto',positive=True,random_state=42,max_iter=5000)
rc.fit(v1, y_train)
pred6=rc.predict(v2)
print("F1 Score: ",f1_score(y_test, pred6,average='macro'))
print(classification_report(pred6,y_test))
print('Confusion Matrix : \n' + str(confusion_matrix(y_test,pred6)))

F1 Score:  0.508411029819048
              precision    recall  f1-score   support

           0       0.58      0.68      0.62       320
           1       0.17      0.35      0.23       332
           2       0.78      0.59      0.67      1634

    accuracy                           0.57      2286
   macro avg       0.51      0.54      0.51      2286
weighted avg       0.67      0.57      0.60      2286

Confusion Matrix : 
[[216  21 136]
 [ 35 115 531]
 [ 69 196 967]]


In [64]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=47)
rf.fit(v1, y_train)
pred9=rf.predict(v2)
print("F1 Score: ",f1_score(y_test, pred9,average='macro'))
print('Confusion Matrix : \n' + str(confusion_matrix(y_test,pred9)))
print(classification_report(pred9,y_test))

F1 Score:  0.5447527631542585
Confusion Matrix : 
[[ 260   21   92]
 [  20   86  575]
 [  37  170 1025]]
              precision    recall  f1-score   support

           0       0.70      0.82      0.75       317
           1       0.13      0.31      0.18       277
           2       0.83      0.61      0.70      1692

    accuracy                           0.60      2286
   macro avg       0.55      0.58      0.54      2286
weighted avg       0.73      0.60      0.65      2286



In [65]:
from sklearn.linear_model import TheilSenRegressor

reg = TheilSenRegressor(random_state=0).fit(v1.toarray(), y_train)
pred6=rc.predict(v2)
print('Accuracy:', accuracy_score(y_test, pred6))
print("F1 Score: ",f1_score(y_test, pred6,average='macro'))
print(classification_report(y_test,pred6))
print('Confusion Matrix : \n' + str(confusion_matrix(y_test,pred6)))

Accuracy: 0.5678040244969379
F1 Score:  0.508411029819048
              precision    recall  f1-score   support

           0       0.68      0.58      0.62       373
           1       0.35      0.17      0.23       681
           2       0.59      0.78      0.67      1232

    accuracy                           0.57      2286
   macro avg       0.54      0.51      0.51      2286
weighted avg       0.53      0.57      0.53      2286

Confusion Matrix : 
[[216  21 136]
 [ 35 115 531]
 [ 69 196 967]]


In [67]:
from sklearn.ensemble import VotingClassifier
voting = VotingClassifier(
estimators=[('rf',rf),("knn",knn),('rc',rc),('dt',clf)],
voting='hard')
voting.fit(v1,y_train)
pred_final=voting.predict(v2)
print('Accuracy:', accuracy_score(y_test, pred_final))
print("F1 Score: ",f1_score(y_test, pred_final,average='macro'))

Accuracy: 0.6041119860017498
F1 Score:  0.5573244389559057


In [87]:
import pickle
modelpkl1 = "Pickle_knn_Model.pkl"  
with open(modelpkl1, 'wb') as file:  
    pickle.dump(knn, file)

In [88]:
import pickle
modelpkl2 = "random_forest_Model.pkl"  
with open(modelpkl2, 'wb') as file:  
    pickle.dump(rf, file)

In [89]:
import pickle
modelpkl3 = "dt.pkl"  
with open(modelpkl3, 'wb') as file:  
    pickle.dump(clf, file)